Source: https://github.com/BlackPianoCat/EasyOpenMM/tree/main

# Imports

In [1]:
import copy
import numpy as np
import time
from utils import *
import openmm as mm
import openmm.unit as u
from tqdm import tqdm
from sys import stdout
from mdtraj.reporters import HDF5Reporter
from openmm.app import PDBFile, PDBxFile, ForceField, Simulation, PDBReporter, PDBxReporter, DCDReporter, StateDataReporter, CharmmPsfFile,  DCDFile
import random
import pyvista as pv
import mdtraj as md
from hilbert import decode, encode
from initial_structures_defs import *
from model import Model

# Initial structure generation functions

# Simulation

In [13]:
# 0. Generate some initial structure
N_beads=100
points = helisa(N_beads)
write_mmcif(points,'init_struct.cif')
generate_psf(N_beads,'LE_init_struct.psf')

# 1. Define System
pdb = PDBxFile('init_struct.cif')
forcefield = ForceField('forcefields/classic_sm_ff.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

# 2. Define the forcefield
# 2.1. Harmonic bond borce between succesive beads
bond_force = mm.HarmonicBondForce()
system.addForce(bond_force)
for i in range(system.getNumParticles() - 1):
    bond_force.addBond(i, i + 1, 0.1, 300000.0)
bond_force.addBond(10, 70, 0.001, 0.001) # connect bead 10 with bead 70

#2.2. Harmonic angle force between successive beads so as to make chromatin rigid
angle_force = mm.HarmonicAngleForce()
system.addForce(angle_force)
for i in range(system.getNumParticles() - 2):
    angle_force.addAngle(i, i + 1, i + 2, np.pi, 0.001)
    
# 3. Minimize energy
simulation = Simulation(pdb.topology, system, integrator)
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('stochastic_LE.dcd', 10))
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=0.001)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('minimized.cif', 'w')) # save minimized file

# 4. Run md simulation
simulation.context.setVelocitiesToTemperature(310, 0)
simulation.step(10000)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.cif', 'w')) # save minimized file
PDBFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.pdb', 'w')) # save minimized file
df = DCDFile(open("after_sim.dcd", "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,107.27322387695312,316.13366567716,169.15917313518588
20,82.96114349365234,320.29261162132025,192.21828011759123
30,89.17096710205078,337.1468686237931,200.8393647802227
40,94.56158447265625,336.2572063356638,195.75287303016904
50,91.58721923828125,357.9594160877168,215.7387975285268
60,82.23358154296875,343.2710475176573,211.4181197788637
70,81.62876892089844,351.0761366188526,218.22942406125458
80,96.01492309570312,356.9387115314603,211.32605065211126
90,87.42857360839844,379.08236931450665,236.2147398432575
100,103.54850006103516,391.5979905053973,233.29555949226028
110,87.23733520507812,406.9284926354885,258.9226154241774
120,89.27845764160156,412.5447427332401,261.81816440326367
130,103.89022827148438,423.42676820605993,258.7973884184208
140,104.93695831298828,419.017266407609,254.37830523289568
150,127.93624877929688,434.52535669505596,248.31107097290794
160,106.31491088867188,454.4865112155676,281

# Visualization

In [14]:
traj = md.load("after_sim.cif")

positions = traj.xyz

mesh = pv.PolyData(positions[0])

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions[0])

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9595e7a00_1&reconnect=auto" class="pyvis…

### Testing initial structure generation function

In [15]:
positions = cube_outer(1000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f95806f820_2&reconnect=auto" class="pyvis…

In [25]:
positions = hilbert_curve3d(4000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

c:\Users\Micha\anaconda3\envs\wb\lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f947e019d0_8&reconnect=auto" class="pyvis…

# Testing multiple initial structures

In [3]:
initial_structure_gen_list = [line,random_walk,random_walk2,hilbert_curve2d, hilbert_curve3d, helisa, sphere, cube, cube_outer]

In [4]:
import os
def make_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

for f in initial_structure_gen_list:
    make_directory_if_not_exists(f'initial_structures_tests/{f.__name__}')

In [29]:
def run_simulation_on_initial_structure(initail_structure_gen_fun, N_beads):    
    points = initail_structure_gen_fun(N_beads)
    write_mmcif(points,f'initial_structures_tests/{initail_structure_gen_fun.__name__}/init_struct.cif')
    generate_psf(N_beads,f'initial_structures_tests/{initail_structure_gen_fun.__name__}/LE_init_struct.psf')

    # 1. Define System
    pdb = PDBxFile(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/init_struct.cif')
    forcefield = ForceField('forcefields/classic_sm_ff.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
    integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

    # 2. Define the forcefield
    # 2.1. Harmonic bond borce between succesive beads
    bond_force = mm.HarmonicBondForce()
    system.addForce(bond_force)
    for i in range(system.getNumParticles() - 1):
        bond_force.addBond(i, i + 1, 0.1, 300000.0)
    bond_force.addBond(10, 70, 0.0001, 100000) # connect bead 10 with bead 70

    #2.2. Harmonic angle force between successive beads so as to make chromatin rigid
    angle_force = mm.HarmonicAngleForce()
    system.addForce(angle_force)
    for i in range(system.getNumParticles() - 2):
        angle_force.addAngle(i, i + 1, i + 2, np.pi, 10)

    # 3. Minimize energy
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
    simulation.reporters.append(DCDReporter(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/stochastic_LE.dcd', 10))
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy(tolerance=0.001)
    state = simulation.context.getState(getPositions=True)
    PDBxFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/minimized.cif', 'w')) # save minimized file

    # 4. Run md simulation
    simulation.context.setVelocitiesToTemperature(310, 0)
    simulation.step(10000)
    state = simulation.context.getState(getPositions=True)
    PDBxFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.cif', 'w')) # save minimized file
    PDBFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.pdb', 'w')) # save minimized file
    df = DCDFile(open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.dcd', "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
    df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

In [4]:
for f in initial_structure_gen_list:
    #run_simulation_on_initial_structure(f,100)
    model = Model(200, 30, f)
    model.run_simulation()

ValueError: Could not locate file "notebooks\forcefields\classic_sm_ff.xml"

In [31]:
plotter = pv.Plotter(shape=(len(initial_structure_gen_list), 2), notebook=True, window_size=(1000,2000))
for i,f in enumerate(initial_structure_gen_list):
    traj = md.load_dcd(f'initial_structures_tests/{f.__name__}/after_sim.dcd', top=f'initial_structures_tests/{f.__name__}/after_sim.pdb')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,1)
    plotter.add_text(f"{f.__name__}, after simulation", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)

    traj = md.load(f'initial_structures_tests/{f.__name__}/init_struct.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,0)
    plotter.add_text(f"{f.__name__}, initial structure", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9584494f0_9&reconnect=auto" class="pyvis…